In [ ]:
# !pip install kaggle
# !pip install pytorch-lightning

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"faissalizii","key":"31236fe96879a3632e37092b52fae4e9"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

 99% 1.32G/1.33G [00:16<00:00, 121MB/s]
100% 1.33G/1.33G [00:16<00:00, 86.5MB/s]


In [ ]:
import os
from PIL import Image
import zipfile
import numpy as np
from tqdm import tqdm

# Path to the zip file containing the CelebA dataset
zip_file_path = "/content/celeba-dataset.zip"

# Function to resize images
def resize_images(zip_file_path, target_size=(64, 64), limit=20000):
    X_train = []
    image_count = 0
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for filename in tqdm(zip_ref.namelist()):
            if image_count >= limit:
                break
            with zip_ref.open(filename) as file:
                # Open the image
                with Image.open(file) as img:
                    # Resize the image
                    resized_img = img.resize(target_size)
                    # Convert image to numpy array and append to X_train
                    X_train.append(np.array(resized_img))
                    image_count += 1
    return X_train

# Resize images and store them in X_train
X_train = resize_images(zip_file_path)

print("Number of resized images:", len(X_train))

FileNotFoundError: [Errno 2] No such file or directory: '/content/celeba-dataset.zip'

In [ ]:
X_train=np.array(X_train)/ 255.0 # images between -1 and 1

In [ ]:
print(np.shape(X_train))

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt
import numpy as np


BATCH_SIZE=100

USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100
print('using device:', device)

using device: cpu


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Assuming X_train is your numpy array with shape (N, 64,64,3)
# Compute the overall mean and standard deviation across all images and channels
mean = np.mean(X_train, axis=(0, 1,2))
std = np.std(X_train, axis=(0, 1, 2))
print("Mean for each channel:", mean)
print("Standard deviation for each channel:", std)



X_train_normalized = transforms.Normalize(mean, std)(torch.from_numpy(X_train).permute(0, 3, 1, 2)) # (N,3,64,64)
# Convert numpy array to PyTorch TensorDataset
dataset = TensorDataset(X_train_normalized)
loader_train = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle= True )

Mean for each channel: [0.50641318 0.42588447 0.38312325]
Standard deviation for each channel: [0.30727777 0.28683807 0.28646142]


In [ ]:
import torch.nn.utils.spectral_norm as spectral_norm
#the First neural network : the Discriminator : the output should be in [0,1] ---> Fake or not Fake
# class Discriminator(nn.Module):
#       def __init__(self):
#           super(Discriminator, self).__init__()

#           # Convolutional layers
#           self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
#           self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#           self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#           #Drop_out
#           self.conv2_drop =nn.Dropout2d()
#           self.conv3_drop =nn.Dropout2d()

#           # Fully connected layer
#           self.fc1 = nn.Linear(128, 32)  #spectral_norm
#           self.fc2 = nn.Linear(32, 1)    #spectral_norm




#       def forward(self , x) :  #N*C*W*H
#           x = F.relu(F.max_pool2d(self.conv1(x), 2))
#           x = F.relu(F.avg_pool2d(self.conv2_drop(self.conv2(x)), 2))
#           x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))

#           # Flatten the tensor so it can be fed into the FC layers
#           N=x.size(0)
#           x=x.view(N,-1) #flattening
#           x = F.relu(self.fc1(x))
#           x=F.dropout1d(x,training=self.training)
#           x=F.sigmoid(self.fc2(x)) #probability --> [0,1]
#           return x


#the First neural network : the Discriminator : the output should be in [0,1] ---> Fake or not Fake
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 256, kernel_size=3, stride=2, padding=1)
        self.leaky_relu1 = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, stride=2, padding=1)
        self.leaky_relu2 = nn.LeakyReLU()
        # self.batch_norm1 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, stride=2, padding=1)
        self.leaky_relu3 = nn.LeakyReLU()
        # self.batch_norm2 = nn.BatchNorm2d(64)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 8 * 8, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        # x = self.batch_norm1(x)
        x = self.conv3(x)
        x = self.leaky_relu3(x)
        # x = self.batch_norm2(x)
        x = self.flatten(x)
        x = F.sigmoid(self.fc(x)) #probability --> [0,1]
        return x


In [ ]:
# The second neural network : the Generator ; the output should be an image with the shape of input data (3,64,64) and values -1,1 (because we already normalized)

# class Generator (nn.Module ):  #"class that inherents from nn.module"
#     def __init__(self, latent_dim = 120) :
#         super(Generator , self).__init__()
#         self.fc = nn.Linear(latent_dim, 192*16*16)  # [N, 192, 16 , 16]
#         self.ct1 = nn.ConvTranspose2d(192, 128, 2, stride=2)
#         self.ct2 = nn.ConvTranspose2d(128, 64, 3, stride=2)
#         self.conv1 = nn.Conv2d(16, 8, kernel_size=7, stride=1)
#         self.conv2 = nn.Conv2d(8, 3, kernel_size=7, stride=1)  # [N, 3, 64, 64]

#     def forward(self , x ):
#         x=F.relu(self.fc(x))
#         x= x.view((-1,192,16,16)) #-1 for N

#         # Upsample (transposed conv)
#         x=F.relu(self.ct1(x))
#         x=F.relu(self.ct2(x))


#         # to [N,3,64,64]  #No need for invariance
#         x= self.conv1(x)
#         x=self.conv2(x)
#         return x

# The second neural network : the Generator ; the output should be an image with the shape of input data (3,64,64) and values -1,1 (because we already normalized)

class Generator(nn.Module):
    def __init__(self, latent_dim = 128):
        super(Generator, self).__init__()
        self.fc = nn.Linear( latent_dim , 8 * 8 * 128)
        # self.batch_norm1 = nn.BatchNorm2d(8 * 8 * 128)
        self.conv1 = nn.ConvTranspose2d(128, 256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.leaky_relu1 = nn.LeakyReLU()
        # self.batch_norm2 = nn.BatchNorm2d(256)
        self.conv2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.leaky_relu2 = nn.LeakyReLU()
        # self.batch_norm3 = nn.BatchNorm2d(128)
        self.conv3 = nn.ConvTranspose2d(128, 3, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 128, 8, 8) #self.batch_norm1(
        x = self.conv1(x)
        x = self.leaky_relu1(x)
        # x = self.batch_norm2(x)
        x = self.conv2(x)
        x = self.leaky_relu2(x)
        # x = self.batch_norm3(x)
        x = self.conv3(x)
        return x



In [ ]:
#My Part : GAN
class GAN(nn.Module) :
      def __init__(self, latent_dim =128, lr = 0.0001) :
        super(GAN, self).__init__()
        self.lr = lr
        self.latent_dim = latent_dim
        #Creating the networks
        self.generator=Generator(latent_dim = self.latent_dim)
        self.discriminator =Discriminator()
        #Random noise to test the image laters
        self.validation_z = torch.randn(20,self.latent_dim, device=device, dtype=dtype)

      def forward(self , z) :
          return self.generator(z)

      def plot_images(self):
          z = self.validation_z  # Ensuring that the noise vectors have the same data type as the model parameters ==> compatibility & numerical stability
          sample_imgs = self.forward(z).cpu()  # Will generate some images
          fig = plt.figure(figsize=(10, 10))  # Adjust the figure size as needed
          for i in range(20):  # Plotting 20 images
              plt.subplot(4, 5, i + 1)
              plt.tight_layout()
              plt.imshow(sample_imgs.detach()[i, 0, :, :])
              plt.title("Generated Data")
              plt.xticks([])
              plt.yticks([])
              plt.axis('off')
          plt.show()




In [ ]:
def train_module(model, g_opt, d_opt, epochs=1):
    model = model.to(device=device,dtype=dtype)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        # Shuffle the dataset for each epoch
        for t, real_imgs in enumerate(loader_train):
              model.train()  # put model to training mode
              real_imgs = real_imgs.to(device=device, dtype=dtype)  # move to device, e.g. GPU

              #sample noise
              z = torch.randn(real_imgs.size(0) , model.latent_dim,device=device, dtype=dtype)
              # training the generator : max log(D(G(z))) z is noise # by maximizing we want the discriminator to return 1 (because they are all fake)
              #Call  the generator
              fake_imgs = model.forward(z)
              y_hat = model.discriminator(fake_imgs)
              y=torch.ones(fake_imgs.size(0),1,device=device, dtype=dtype)  #real labels (they are all fake) ==> outsmarting
              #loss
              g_loss= F.binary_cross_entropy(y_hat , y)
              #Manually
              g_opt.zero_grad()
              g_loss.backward()
              g_opt.step()

              #training the discriminator max log(D(x))+log(1-D(G(z)))
              #How well can it label as real, and also how can it lable the generated as fake.
              fake_imgs= model.forward(z).detach() #because we already calculated them, to avoid the computation graph
              # How well it can  label as real :
              y_hat_real = model.discriminator(real_imgs)
              y_real=torch.ones(real_imgs.size(0) , 1, device=device, dtype=dtype)  #real labels (they are all real)
              y_real=y_real.type_as(real_imgs)
              real_loss =F.binary_cross_entropy(y_hat_real , y_real)
              #How well it can label as fake :
              y_hat_fake = model.discriminator(fake_imgs)
              y_fake=torch.ones(real_imgs.size(0) ,1,device=device, dtype=dtype)  #real labels (they are all fake "0" with (1-D(G(z)) we put "1" )

              fake_loss =F.binary_cross_entropy(y_hat_fake, y_fake)
              #Global loss :
              d_loss = (real_loss+fake_loss)/2
              #Manually
              d_opt.zero_grad()
              d_loss.backward()
              d_opt.step()


              if t % print_every == 0:
                  print('Iteration %d, loss = %.4f' % (t, loss.item()))
        print('epoch', e)
        model.plot_images()





In [ ]:
model = GAN()
model.plot_images()

NameError: name 'dtype' is not defined

In [ ]:
EPOCHS = 100

opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
train_module(model, optimizer, epochs=EPOCHS)